In [ ]:
!git clone https://github.com/facebookresearch/mmf.git
%cd mmf
!pip install --editable .

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!cp "/content/drive/MyDrive/DL_Group_Project/hm_convert.py" /content/mmf/mmf_cli

In [ ]:
!mmf_convert_hm --zip_file='/content/drive/MyDrive/DL_Group_Project/hateful_memes.zip' --password='' --bypass_checksum=1

In [ ]:
import matplotlib.pyplot as plt
import requests

from PIL import Image

from mmf.models.mmbt import MMBT

model = MMBT.from_pretrained("mmbt.hateful_memes.images")

In [ ]:
# Get prediction on Memotion_Memes dataset

import pandas as pd

ids = []
images_url = []
texts = []
predictions = []
confidences = []

info = pd.read_csv("/content/drive/MyDrive/DL_Group_Project/reference.csv")
txt_info = pd.read_csv("/content/drive/MyDrive/DL_Group_Project/labels.csv")
ids = info.image_name.to_list()
images_url = info.image_url.to_list()
texts = txt_info.text_corrected.to_list()

for i in range(len(ids)):
    image_url = images_url[i] #@param {type:"string"}
    text = texts[i] #@param {type: "string"}
    try:
        output = model.classify(image_url, text)
        predictions.append(output["label"])
        confidences.append(output["confidence"])
    except:
        predictions.append(0)
        confidences.append(0)

In [ ]:
import json

# format pseudo-labeled data and add to hateful_memes dataset for training
info_list = []
info_list1 = [] #confident prediction

id_start = 1198764
# for image file name replacement
name_change = {}

for i in range(len(ids)):
    new_info = {}
    id_start += 1
    new_info['id'] = str(id_start)
    postfix = images_url[i].split('.')[-1]
    new_info['img'] = "img/" + str(id_start) + ".png"
    name_change[ids[i] + '.' + postfix] = str(id_start) + '.png'
    new_info['label'] = predictions[i]
    new_info['text'] = texts[i]
    info_list.append(new_info)

    # change label according to confidence
    if predictions[i] == 0 and confidences[i] < 0.8:
        new_info['label'] = 1
        info_list1.append(new_info)
    elif predictions[i] == 1 and confidences[i] < 0.5:
        new_info['label'] = 0
        info_list1.append(new_info)
    else:
        info_list1.append(new_info)

with open("/content/drive/MyDrive/DL_Group_Project/memotion_pred.jsonl", 'w') as outfile:
    for entry in info_list:
        json.dump(entry, outfile)
        outfile.write('\n')
with open("/content/drive/MyDrive/DL_Group_Project/memotion_pred1.jsonl", 'w') as outfile:
    for entry in info_list1:
        json.dump(entry, outfile)
        outfile.write('\n')
		
# save information for image filename change to json
import json
with open('/content/drive/MyDrive/DL_Group_Project/memotion_name_change.json', 'w') as outfile:
    json.dump(name_change, outfile)

In [ ]:
# Get prediction on Reddit_Memes dataset

import os
import json

ids = []
images_url = []
texts = []
predictions = []
confidences = []
# get texts from json
with open ("/content/drive/MyDrive/DL_Group_Project/reddit_memes_db.json", 'r', encoding="utf8") as f:
    info = json.loads(f.read())

for i in range(1, 3227):
    ids.append(info['_default'][str(i)]['id'])
    images_url.append(info['_default'][str(i)]['media'])
    texts.append(info['_default'][str(i)]['title'])

for i in range(3226):
    image_url = images_url[i] #@param {type:"string"}
    text = texts[i] #@param {type: "string"}
    try:
        output = model.classify(image_url, text)
        predictions.append(output["label"])
        confidences.append(output["confidence"])
    except:
        predictions.append(0)
        confidences.append(0)

In [ ]:
# format pseudo-labeled data and add to hateful_memes dataset for training
info_list = []
info_list1 = [] #confident prediction

id_start = 198764
# for image file name replacement
name_change = {}

for i in range(3226):
    new_info = {}
    id_start += 1
    new_info['id'] = str(id_start)
    postfix = images_url[i].split('.')[-1]
    new_info['img'] = "img/" + str(id_start) + ".png"
    name_change[ids[i] + '.' + postfix] = str(id_start) + '.png'
    new_info['label'] = predictions[i]
    new_info['text'] = texts[i]
    info_list.append(new_info)

    # change label according to confidence
    if predictions[i] == 0 and confidences[i] < 0.9:
        new_info['label'] = 1
        info_list1.append(new_info)
    elif predictions[i] == 1 and confidences[i] < 0.5:
        new_info['label'] = 0
        info_list1.append(new_info)
    else:
        info_list1.append(new_info)

with open("/content/drive/MyDrive/DL_Group_Project/train.jsonl", 'w') as outfile:
    for entry in info_list:
        json.dump(entry, outfile)
        outfile.write('\n')
with open("/content/drive/MyDrive/DL_Group_Project/train1.jsonl", 'w') as outfile:
    for entry in info_list1:
        json.dump(entry, outfile)
        outfile.write('\n')
		
# save information for image filename change to json
import json
with open('/content/drive/MyDrive/DL_Group_Project/name_change.json', 'w') as outfile:
    json.dump(name_change, outfile)